In [117]:
# pip install --upgrade transformers

In [1]:
import torch
import transformers
import accelerate
import huggingface_hub

print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)
print("Huggingface Hub:", huggingface_hub.__version__)


Torch: 2.2.2
Transformers: 4.49.0
Accelerate: 1.4.0
Huggingface Hub: 0.29.1


In [1]:
import pandas as pd
import torch
import json
import ast
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report
from datasets import load_dataset

from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch.multiprocessing as mp

In [19]:
# # Load Dataset
# dataset = load_dataset("NebulaByte/E-Commerce_Customer_Support_Conversations")
# df = pd.DataFrame(dataset['train'])


In [25]:
# df['issue_area'].value_counts()

In [26]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from datasets import load_dataset

# # Load the dataset from Hugging Face and convert to DataFrame
# dataset = load_dataset("NebulaByte/E-Commerce_Customer_Support_Conversations")
# df = pd.DataFrame(dataset['train'])

# # Check the original class distribution
# print("Original distribution:")
# print(df['issue_area'].value_counts())

# # Perform a stratified split into 512 and 488 samples
# train_df, inference_df = train_test_split(
#     df, 
#     test_size=488, 
#     stratify=df['issue_area'], 
#     random_state=42
# )

# # Verify the split sizes
# print("\nTraining set shape:", train_df.shape)   # Expected ~512 rows
# print("Inference set shape:", inference_df.shape)  # Expected ~488 rows




In [24]:
# # Save the CSV files
# train_df.to_csv("train_512.csv", index=False)
# inference_df.to_csv("inference_488.csv", index=False)

In [27]:
# df=pd.read_csv("train_512.csv")

In [33]:
# df.head(2)

In [34]:
# df['issue_area'].value_counts()

In [35]:
# import pandas as pd
# import numpy as np

# # Set the total target sample count and compute the per-class target
# total_target = 512
# num_classes = df['issue_area'].nunique()  # Number of unique issue areas
# target_per_class = total_target // num_classes  # For example, if 6 classes then 512 // 6 = 85 per class
# remainder = total_target % num_classes           # Remainder samples (e.g., 512 - 85*6 = 2)

# # Create a list to hold sampled data for each class
# balanced_samples = []

# # Group the DataFrame by issue_area and sample target_per_class rows for each group
# for issue, group in df.groupby('issue_area'):
#     # If the group has fewer rows than target_per_class, sample with replacement (oversampling)
#     if len(group) < target_per_class:
#         sampled = group.sample(n=target_per_class, replace=True, random_state=42)
#     else:
#         sampled = group.sample(n=target_per_class, replace=False, random_state=42)
#     balanced_samples.append(sampled)

# # Concatenate all the sampled groups
# balanced_df = pd.concat(balanced_samples).reset_index(drop=True)

# # If there is a remainder, randomly sample extra rows from randomly chosen groups
# if remainder > 0:
#     extra_samples = []
#     extra_classes = np.random.choice(df['issue_area'].unique(), remainder, replace=False)
#     for cls in extra_classes:
#         group = df[df['issue_area'] == cls]
#         extra_sample = group.sample(n=1, replace=True, random_state=42)
#         extra_samples.append(extra_sample)
#     balanced_df = pd.concat([balanced_df] + extra_samples).reset_index(drop=True)

# # Shuffle the final balanced DataFrame
# balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# # Display the new balanced distribution
# print("\nBalanced distribution:")
# print(balanced_df['issue_area'].value_counts())



In [32]:
# # Save the balanced DataFrame to a CSV file
# balanced_df.to_csv("balanced_512.csv", index=False)

In [36]:
df=pd.read_csv("balanced_512.csv")

In [37]:
df['issue_area'].value_counts()

issue_area
Warranty                     86
Shopping                     86
Shipping                     85
Login and Account            85
Order                        85
Cancellations and returns    85
Name: count, dtype: int64

In [39]:
df.head(1)

,issue_area,issue_category,issue_sub_category,issue_category_sub_category,customer_sentiment,product_category,product_sub_category,issue_complexity,agent_experience_level,agent_experience_level_desc,conversation
0,Shipping,Free Delivery Qualification,Reasons for an order not qualifying for free d...,Free Delivery Qualification -> Reasons for an ...,neutral,Men/Women/Kids,Diaper,medium,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for calling BrownBox Customer...


In [40]:
def preprocess(conversation):
    # Ensure conversation is a list
    if isinstance(conversation, list):
        return " ".join([turn.get('text', '') for turn in conversation if isinstance(turn, dict)])
    return ""

In [41]:
# If conversations are stored as lists of dictionaries
if isinstance(df['conversation'].iloc[0], list):
    # If conversation is a list of dictionaries, preprocess it
    df['Text'] = df['conversation'].apply(preprocess)
else:
    # If conversation is already plain text, just convert it to string
    df['Text'] = df['conversation'].apply(lambda x: str(x))


In [42]:
df['issue_area'].unique()

array(['Shipping', 'Warranty', 'Login and Account', 'Order', 'Shopping',
       'Cancellations and returns'], dtype=object)

In [43]:
df['issue_area'].value_counts()

issue_area
Warranty                     86
Shopping                     86
Shipping                     85
Login and Account            85
Order                        85
Cancellations and returns    85
Name: count, dtype: int64

In [44]:
# Encode labels
df['Label'] = df['issue_area'].astype('category').cat.codes

In [45]:
df['Label'].unique()

array([3, 5, 1, 2, 4, 0], dtype=int8)

In [12]:
# for name, module in model.named_modules():
#     print(name)


In [46]:
# Split data
train_texts, test_texts, train_labels, test_labels = train_test_split(df['Text'], df['Label'], test_size=0.2, random_state=42)
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Update tokenizer max_length if needed
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=256)


train_labels = torch.tensor(train_labels.tolist())
test_labels = torch.tensor(test_labels.tolist())

In [47]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [48]:
model_name = "bert-base-uncased"  # Change model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(df['Label'].unique()))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
# Compute class weights and define weighted loss (if using a custom Trainer)
from sklearn.utils.class_weight import compute_class_weight
from torch.nn import CrossEntropyLoss

class_weights = compute_class_weight("balanced", classes=np.unique(train_labels.numpy()), y=train_labels.numpy())
class_weights = torch.tensor(class_weights, dtype=torch.float)
loss_function = CrossEntropyLoss(weight=class_weights)

In [50]:
# Define a Custom Trainer that uses the weighted loss
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = loss_function(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [51]:
# Update LoRA config for increased capacity
lora_config = LoraConfig(
    r=32,                # Increased rank
    lora_alpha=64,       # Increased scaling factor
    lora_dropout=0.1,    # Reduced dropout
    bias="none",
    target_modules=["query", "value"]  # You can experiment with adding "key" as well
)
peft_model = get_peft_model(model, lora_config)

In [52]:

# Print trainable vs total parameters to check freezing is in effect
trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in peft_model.parameters())
print(f"Trainable parameters: {trainable_params} / Total parameters: {total_params}")


Trainable parameters: 1179648 / Total parameters: 110666502


In [53]:
# ---------------------------
# Setup Training Arguments for CPU
# ---------------------------
mp.set_start_method("fork", force=True)

# Update training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    logging_dir='./logs',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=30,      # Increased epochs
    learning_rate=1e-4,       # Lower learning rate
    weight_decay=0.01,
    warmup_ratio=0.1,
    logging_steps=10,
    report_to="none"
)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [54]:
def compute_metrics(pred):
    preds = pred.predictions.argmax(-1)
    labels = pred.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


In [55]:
import torch.nn as nn

class WrappedPeftModel(nn.Module):
    def __init__(self, peft_model):
        super().__init__()
        self.peft_model = peft_model

    def forward(self, input_ids, attention_mask=None, labels=None, **kwargs):
        # Remove unexpected keyword arguments that the underlying model doesn't accept.
        kwargs.pop("num_items_in_batch", None)
        return self.peft_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, **kwargs)

wrapped_model = WrappedPeftModel(peft_model)



In [56]:
# ---------------------------
# (Optional) Subclass Trainer for Custom Loss
# ---------------------------
# If using a custom loss (e.g., weighted loss), you can override compute_loss.
# For now, we assume the model's forward computes loss when labels are provided.
# Otherwise, create a CustomTrainer that overrides compute_loss.


# ---------------------------
# Train and Evaluate
# ---------------------------
trainer = Trainer(
    model=wrapped_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)



In [47]:
trainer.train()


# Evaluate
preds_output = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(preds_output.predictions), dim=1)

print(classification_report(test_labels, preds))


# Print overall accuracy as a percentage
overall_accuracy = accuracy_score(test_labels, preds)
print("Total Accuracy: {:.2f}%".format(overall_accuracy * 100))

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.785600,1.791318,0.262136,0.147320,0.262136,0.187967
2,1.663400,1.709562,0.320388,0.102649,0.320388,0.155483
3,1.695500,1.670120,0.310680,0.136373,0.310680,0.167903
4,1.660200,1.658329,0.330097,0.206623,0.330097,0.175532
5,1.418100,1.490245,0.466019,0.364771,0.466019,0.358519
6,1.392800,1.394375,0.524272,0.528086,0.524272,0.462914
7,1.374800,1.333436,0.592233,0.602482,0.592233,0.541146
8,1.302600,1.275894,0.611650,0.594333,0.611650,0.569737
9,1.203800,1.224077,0.640777,0.621939,0.640777,0.600551
10,1.080000,1.161676,0.679612,0.651872,0.679612,0.634249


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

              precision    recall  f1-score   support

           0       0.72      0.88      0.79        33
           1       0.94      0.94      0.94        16
           2       0.46      0.62      0.53        21
           3       1.00      0.36      0.53        11
           4       1.00      0.18      0.31        11
           5       0.85      1.00      0.92        11

    accuracy                           0.72       103
   macro avg       0.83      0.66      0.67       103
weighted avg       0.78      0.72      0.70       103

Total Accuracy: 71.84%


In [57]:
trainer.train()


# Evaluate
preds_output = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(preds_output.predictions), dim=1)

print(classification_report(test_labels, preds))


# Print overall accuracy as a percentage
overall_accuracy = accuracy_score(test_labels, preds)
print("Total Accuracy: {:.2f}%".format(overall_accuracy * 100))

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.811000,1.814481,0.155340,0.075795,0.155340,0.095065
2,1.765500,1.769476,0.194175,0.152029,0.194175,0.143831
3,1.727900,1.644919,0.339806,0.266075,0.339806,0.254828
4,1.597600,1.493049,0.427184,0.372211,0.427184,0.363640
5,1.331500,1.406126,0.553398,0.466683,0.553398,0.495554
6,1.348900,1.349473,0.582524,0.648531,0.582524,0.559239
7,1.356300,1.289282,0.718447,0.786408,0.718447,0.689438
8,1.311500,1.236338,0.766990,0.837795,0.766990,0.771695
9,1.211300,1.201824,0.776699,0.809482,0.776699,0.773782
10,1.168200,1.163892,0.766990,0.810960,0.766990,0.764446


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

              precision    recall  f1-score   support

           0       0.94      0.79      0.86        19
           1       0.90      1.00      0.95        18
           2       0.75      0.67      0.71        18
           3       0.75      1.00      0.86        15
           4       0.93      0.82      0.88        17
           5       1.00      1.00      1.00        16

    accuracy                           0.87       103
   macro avg       0.88      0.88      0.87       103
weighted avg       0.88      0.87      0.87       103

Total Accuracy: 87.38%


In [72]:
# This saves the adapter weights and configuration to the specified directory.
# peft_model.save_pretrained("./exp_4_87_30epc_saved_model")

In [61]:
inference_df.head()

,issue_area,issue_category,issue_sub_category,issue_category_sub_category,customer_sentiment,product_category,product_sub_category,issue_complexity,agent_experience_level,agent_experience_level_desc,conversation
0,Login and Account,Account Reactivation and Deactivation,Reactivating an inactive account,Account Reactivation and Deactivation -> React...,negative,Appliances,Microwave Oven,less,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for calling BrownBox customer...
1,Warranty,Product Registration and Warranty,Need to register the product with the brand fo...,Product Registration and Warranty -> Need to r...,neutral,Appliances,Electric Kettle,medium,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox customer...
2,Order,Order Confirmation and Status,Tracking/Shipping Updates,Order Confirmation and Status -> Tracking/Ship...,neutral,Appliances,Air Cooler,less,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for calling BrownBox Customer...
3,Order,Order Confirmation and Status,Confirming order status,Order Confirmation and Status -> Confirming or...,positive,Electronics,Smart Band,less,experienced,"confidently handles complex customer issues, e...","Customer: Hi, I'm calling to inquire about my ..."
4,Cancellations and returns,Pickup and Shipping,Pickup process,Pickup and Shipping -> Pickup process,neutral,Electronics,Tablet,less,junior,"handles customer inquiries independently, poss...","Customer: Hi, I would like to know about the p..."


In [68]:
import pandas as pd

# Load the inference data from CSV
inference_df = pd.read_csv("inference_488.csv")

# Assuming the conversation text is stored in a column named "Text"
# (adjust the column name if necessary)
sample_conversations = inference_df["conversation"].tolist()

# Create an inverted mapping: string to numeric
inverted_label_mapping = {v: k for k, v in label_mapping.items()}

# Optionally, print the first few rows to verify the data
# print(inference_df.head())

# If you have a label mapping dictionary defined (from training), e.g.:
# label_mapping = dict(enumerate(df['issue_area'].astype('category').cat.categories))

label_mapping= { 
                 0: 'Cancellations and returns',
                 1: 'Login and Account',
                 2: 'Order',
                 3: 'Shipping',
                 4: 'Shopping',
                 5: 'Warranty'
                }

# Run inference on each sample conversation using your wrapped model
# Note: Make sure that wrapped_model and tokenizer have been loaded (as during training)
predictions = []
for idx, conversation in enumerate(sample_conversations):
    predicted_label = infer_intent(conversation, wrapped_model, tokenizer, max_length=256, device="cpu")
    predictions.append(predicted_label)
    # print(f"Conversation {idx+1}:")
    # print(conversation)
    # print("Predicted Label:", predicted_label)
    # print("Predicted Intent:", label_mapping.get(predicted_label, "Unknown"))
    # print("-" * 80)

# Optionally, add the predictions back to the DataFrame and save to a new CSV
inference_df["Predicted_Label"] = predictions
inference_df["Predicted_Intent"] = inference_df["Predicted_Label"].map(label_mapping)
# inference_df.to_csv("inference_with_predictions.csv", index=False)


In [65]:
dict(enumerate(df['issue_area'].astype('category').cat.categories))

{0: 'Cancellations and returns',
 1: 'Login and Account',
 2: 'Order',
 3: 'Shipping',
 4: 'Shopping',
 5: 'Warranty'}

In [73]:
# Create an inverted mapping: string to numeric
# inverted_label_mapping = {v: k for k, v in label_mapping.items()}
# Convert the ground truth issue areas to numeric using the inverted mapping
ground_truth_numeric = inference_df["issue_area"].map(inverted_label_mapping)

# Compute overall accuracy
acc = accuracy_score(ground_truth_numeric, predictions)
print("Total Accuracy: {:.2f}%".format(acc * 100))


Total Accuracy: 83.61%


In [70]:
inference_df['issue_area'].value_counts()

issue_area
Cancellations and returns    139
Order                        132
Login and Account             74
Shopping                      57
Warranty                      51
Shipping                      35
Name: count, dtype: int64

In [71]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel

def infer_intent(conversation_text, model, tokenizer, max_length=256, device="cpu"):
    """
    Performs inference on a single conversation text.
    
    Args:
      conversation_text (str): The raw conversation text.
      model: The trained LoRA-wrapped model.
      tokenizer: The tokenizer used during training.
      max_length (int): Maximum sequence length (must match training).
      device (str): Device to run inference on (e.g., "cpu").
      
    Returns:
      predicted_label (int): The predicted label index.
    """
    # Ensure model is on the right device and in eval mode
    model.to(device)
    model.eval()
    
    # Tokenize the input text
    inputs = tokenizer(
        conversation_text,
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    )
    # Move inputs to the same device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Disable gradients for inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=-1).item()
    return predicted_label

# Sample conversation data for inference
sample_conversations = [

    """Agent: Thank you for calling XYZ customer support. My name is Alex. How may I assist you today?
Customer: Hi Alex, I placed an order two weeks ago and haven't received it yet. Could you check its status for me?
Agent: Certainly. May I have your order number, please?
Customer: Yes, it's 123456.
Agent: Thank you. I see that there is a delay due to logistics. We are working to resolve it and will update you shortly.""",


    """Agent: Hello, thank you for contacting our support. This is Lisa speaking. How can I help you today?
Customer: Hi Lisa, I would like to cancel my order as I no longer need the product.
Agent: I’m sorry to hear that. Could you please provide your order number?
Customer: It’s 654321.
Agent: Thank you. I have processed the cancellation and you will receive a refund within 3-5 business days.""",


    """Agent: Good day, thank you for calling our support center. How may I assist you?
Customer: Hello, I received an email about a shipping delay, but I haven’t seen any update on my package status.
Agent: I apologize for the inconvenience. Could you please share your tracking number?
Customer: Sure, it's ABC123XYZ.
Agent: Thank you. It appears that there was a delay at the distribution center. Your package should arrive soon."""
]

# Example: Run inference on each sample conversation
for idx, conversation in enumerate(sample_conversations):
    predicted_label = infer_intent(conversation, wrapped_model, tokenizer, max_length=256, device="cpu")
    
    # If you have a mapping from label indices to intent names, you can display the intent.
    # For example, if you computed the mapping during preprocessing:
    label_mapping = dict(enumerate(df['issue_area'].astype('category').cat.categories))

    print(f"Conversation {idx+1}:")
    print(conversation)
    print("Predicted Label:", predicted_label)
    print("Predicted Intent:", label_mapping.get(predicted_label, "Unknown"))
    print("-" * 80)


Conversation 1:
Agent: Thank you for calling XYZ customer support. My name is Alex. How may I assist you today?
Customer: Hi Alex, I placed an order two weeks ago and haven't received it yet. Could you check its status for me?
Agent: Certainly. May I have your order number, please?
Customer: Yes, it's 123456.
Agent: Thank you. I see that there is a delay due to logistics. We are working to resolve it and will update you shortly.
Predicted Label: 2
Predicted Intent: Order
--------------------------------------------------------------------------------
Conversation 2:
Agent: Hello, thank you for contacting our support. This is Lisa speaking. How can I help you today?
Customer: Hi Lisa, I would like to cancel my order as I no longer need the product.
Agent: I’m sorry to hear that. Could you please provide your order number?
Customer: It’s 654321.
Agent: Thank you. I have processed the cancellation and you will receive a refund within 3-5 business days.
Predicted Label: 2
Predicted Intent: